In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [4]:
portfolio = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Portfolio")
TenYearYield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="10yUST Yields")
HYyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HY Index")
IGyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="IG Index")
MoveIdx = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="MOVE Index")
Vix = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="VIX")

portfolio = portfolio.set_index("Date")
TenYearYield = TenYearYield.set_index("Date").sort_index()
HYyield = HYyield.set_index("Date").sort_index()
IGyield = IGyield.set_index("Date").sort_index()
MoveIdx = MoveIdx.set_index("Date").sort_index()
Vix = Vix.set_index("Date").sort_index()

portfolio.loc[:,"TLTCumDiv"]=portfolio.loc[:,"TLT Dividends"][::-1].cumsum()[::-1]
portfolio.loc[:,"TotalValueTLT"]=portfolio.loc[:,"TLT Position"]+portfolio.loc[:,"TLTCumDiv"]
portfolio = portfolio.sort_index()

TLTCorrData = pd.DataFrame(portfolio.loc["2021-01-11":"2025-12-31","TotalValueTLT"]).merge(HYyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
TLTCorrData.loc[:,"TotalReturnsTLT"] = TLTCorrData["TotalValueTLT"].pct_change()
TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(HYyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"HY_Index_OAS","INDEX_Z_SPREAD_BP":"HY_INDEX_Z_SPREAD"})
TLTCorrData.loc[:,"HY_Index_OAS_Diff"]=TLTCorrData["HY_Index_OAS"].diff()
TLTCorrData.loc[:,"HY_INDEX_Z_SPREAD_Diff"]=TLTCorrData["HY_INDEX_Z_SPREAD"].diff()

TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(TenYearYield.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"PX_LAST":"10y_UST_INDEX_PX"})
TLTCorrData.loc[:,"10yYieldDiff"] = TLTCorrData["10y_UST_INDEX_PX"].diff()

TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"IG_Index_OAS"})
TLTCorrData.loc[:,"IG_Index_OAS_Diff"]=TLTCorrData["IG_Index_OAS"].diff()
TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"INDEX_Z_SPREAD_BP":"IG_INDEX_Z_SPREAD"})
TLTCorrData.loc[:,"IG_Index_Z_SPREAD_Diff"]=TLTCorrData["IG_INDEX_Z_SPREAD"].diff()

TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(portfolio.loc["2021-01-11":"2025-12-31","SPY Position"]/-10,left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"SPY Position":"SPY_LAST_PX"})
TLTCorrData.loc[:,"ReturnSPY"] = TLTCorrData["SPY_LAST_PX"].pct_change()

TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(MoveIdx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"PX_LAST":"MOVE Idx"})

TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(Vix.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"PX_LAST":"Vix Idx"})

TLTCorrData=TLTCorrData.dropna()
TLTCorrData = TLTCorrData.sort_index(ascending=False)

In [8]:
TLTCorrData.corr()

,TotalValueTLT,HY_Index_OAS,TotalReturnsTLT,HY_INDEX_Z_SPREAD,HY_Index_OAS_Diff,HY_INDEX_Z_SPREAD_Diff,10y_UST_INDEX_PX,10yYieldDiff,IG_Index_OAS,IG_Index_OAS_Diff,IG_INDEX_Z_SPREAD,IG_Index_Z_SPREAD_Diff,SPY_LAST_PX,ReturnSPY,MOVE Idx,Vix Idx,PC1,PC2,PC3
TotalValueTLT,1.000000,0.163862,-0.010064,0.106484,-0.012076,-0.011490,0.983071,-0.001206,0.104117,-0.038606,0.106429,-0.039228,0.443830,0.010833,0.561815,-0.232400,4.001694e-01,2.007647e-01,3.396564e-01
HY_Index_OAS,0.163862,1.000000,-0.009699,0.996400,0.063370,0.063636,0.076843,-0.013410,0.962595,0.055573,0.962509,0.055296,-0.660061,0.024467,0.746043,0.495512,5.383689e-01,2.528939e-01,4.511988e-01
TotalReturnsTLT,-0.010064,-0.009699,1.000000,-0.010531,0.041067,0.030761,0.042016,0.002640,-0.026340,0.007064,-0.026488,0.006401,0.028000,0.060812,0.005808,-0.047155,4.641241e-02,-1.827113e-02,-3.523278e-02
HY_INDEX_Z_SPREAD,0.106484,0.996400,-0.010531,1.000000,0.064887,0.067203,0.017809,-0.019320,0.963198,0.057117,0.963278,0.057197,-0.680896,0.025853,0.722940,0.523304,5.231664e-01,2.389320e-01,4.393158e-01
HY_Index_OAS_Diff,-0.012076,0.063370,0.041067,0.064887,1.000000,0.983724,-0.013144,-0.269423,0.031004,0.751647,0.031223,0.751890,-0.025722,-0.002172,0.053103,0.184556,3.463033e-02,-2.068506e-01,1.791443e-01
HY_INDEX_Z_SPREAD_Diff,-0.011490,0.063636,0.030761,0.067203,0.983724,1.000000,-0.013319,-0.319092,0.031865,0.746788,0.032067,0.749716,-0.026017,-0.007087,0.054464,0.182251,3.198016e-02,-2.463924e-01,2.097602e-01
10y_UST_INDEX_PX,0.983071,0.076843,0.042016,0.017809,-0.013144,-0.013319,1.000000,0.002860,0.019554,-0.038979,0.021764,-0.039927,0.516161,0.014785,0.522984,-0.280710,3.757982e-01,1.884973e-01,3.111456e-01
10yYieldDiff,-0.001206,-0.013410,0.002640,-0.019320,-0.269423,-0.319092,0.002860,1.000000,0.018142,-0.190721,0.017468,-0.205694,-0.033513,-0.019487,0.019557,0.048706,3.696598e-03,8.573729e-01,-5.146824e-01
IG_Index_OAS,0.104117,0.962595,-0.026340,0.963198,0.031004,0.031865,0.019554,0.018142,1.000000,0.045124,0.999928,0.044353,-0.702706,0.008197,0.753810,0.485665,5.325811e-01,2.883885e-01,4.489827e-01
IG_Index_OAS_Diff,-0.038606,0.055573,0.007064,0.057117,0.751647,0.746788,-0.038979,-0.190721,0.045124,1.000000,0.045625,0.998390,-0.032072,-0.010115,0.071133,0.215286,4.198287e-02,-1.309487e-01,1.527244e-01


In [11]:
portfolio["TLTCumDiv"]

Date
2021-01-11     -0.195560
2021-01-12     -0.391920
2021-01-13     -0.588382
2021-01-14     -0.786316
2021-01-15     -0.983974
                 ...    
2025-12-24   -387.011360
2025-12-26   -387.419253
2025-12-29   -387.419253
2025-12-30   -387.419253
2025-12-31   -387.419253
Name: TLTCumDiv, Length: 1250, dtype: float64

In [10]:
TLTCorrData[["10yYieldDiff","TotalReturnsTLT"]]

,10yYieldDiff,TotalReturnsTLT
Date,,
2025-12-31,0.0451,-0.004599
2025-12-30,0.0117,0.004962
2025-12-29,-0.0175,-0.001748
2025-12-26,-0.0058,0.004354
2025-12-24,-0.0293,-0.001109
...,...,...
2021-01-19,0.0051,-0.001836
2021-01-15,-0.0457,0.011836
2021-01-14,0.0460,0.003289


In [7]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(TLTCorrData[["10yYieldDiff","MOVE Idx","ReturnSPY"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["10yYieldDiff","MOVE Idx","ReturnSPY"]))

X_pca = pcav1.transform(X_Std)

TLTCorrData.loc[:,"PC1"]=X_pca[:,0]
TLTCorrData.loc[:,"PC2"]=X_pca[:,1]
TLTCorrData.loc[:,"PC3"]=X_pca[:,2]
#TLTCorrData.loc[:,"PC4"]=X_pca[:,3]
#TLTCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(TLTCorrData[["PC1","PC2","PC3"]])
Y = TLTCorrData["TotalReturnsTLT"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.3424918  0.33903317 0.31847503]
   10yYieldDiff  MOVE Idx  ReturnSPY
0      0.003647  0.708396   0.705805
1      0.850135  0.369452  -0.375201
2     -0.526552  0.601398  -0.600886


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsTLT   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.542
Date:                Thu, 15 Jan 2026   Prob (F-statistic):              0.202
Time:                        00:15:33   Log-Likelihood:                 4007.1
No. Observations:                1240   AIC:                            -8006.
Df Residuals:                    1236   BIC:                            -7986.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003      0.000     -0.966      0.334      -0.001       0.000
PC1            0.0004      0.000      1.635      0.102   -8.77e-05       0.001
PC2           -0.0002      0.000     -0.644      0.520      -0.001       0.000
PC3           -0.0003      0.000     -1.241      0.215      -0.001       0.000
==============================================================================
Omnibus:                        9.800   Durbin-Watson:                   2.107
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               13.513
Skew:                           0.054   Prob(JB):                      0.00116
Kurtosis:                       3.500   Cond. No.                         1.04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""